In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('data/wine.csv')

In [3]:
dataset.head()

,Alcohol real,Malic_acid real,Ash real,Alcalinity_of_ash real,Magnesium real,Total_phenols real,Flavanoids real,Nonflavanoid_phenols real,Proanthocyanins real,Color_intensity real,Hue real,OD280/OD315_of_diluted_wines real,Proline real,Class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

In [5]:
model = GaussianNB()

In [6]:
X = dataset.drop(columns=['Class'])
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)   

In [7]:
y = dataset['Class']

In [8]:
from sklearn.model_selection import cross_val_score

In [9]:
import numpy as np

In [10]:
score = np.mean(cross_val_score(model, X, y, cv=10))

In [11]:
score

0.9616959064327485

In [12]:
from sklearn.neural_network import MLPClassifier

In [13]:
net = MLPClassifier(solver='lbfgs', alpha=1e-4, hidden_layer_sizes=(15, 5), random_state=1)

In [14]:
score = np.mean(cross_val_score(net, X, y, cv=10))

In [15]:
score

0.983625730994152

In [16]:
from sklearn.svm import LinearSVC

In [17]:
linear_svm = LinearSVC(C=0.05)

In [18]:
score = np.mean(cross_val_score(linear_svm, X, y, cv=10))

In [19]:
score

0.9833333333333334